<a href="https://colab.research.google.com/github/LuisaFerro/Adaptador-de-PDF/blob/main/trabIST_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Antes de iniciar o código, dê play nessa célula
#Se der algum problema no código, vá em Ambiente de Execução > Reiniciar sessão, e dê play nessa célula novamente. Só então, dê play na célula abaixo

!pip -q install google-generativeai==0.3.0
!pip -q install google-ai-generativelanguage==0.4.0
!pip install --upgrade google-generativeai
!apt-get install poppler-utils
!pip install reportlab

!curl -o questao_listaUNb.png https://drive.google.com/uc?export=view&id=15DZT3vHPOOKs06cyPUWjATMnq9aljf8u
!curl -o questão3.png https://drive.google.com/uc?export=view&id=13Bt0G4MNHn_yFiDAMLRn2MpAUFxqGwmx


!curl -o Teste1.png https://drive.google.com/uc?export=18GMKkBPAx5zqpC_LtsJ-mtX0K3Hd6Pfk
!curl -o Teste2.png https://drive.google.com/uc?export=18zgZaV38zB2AnfA522lmCzNIaDhxqSDv
!curl -o Teste3.png https://drive.google.com/uc?export=14iP3US38CSWOZxXVjmsnOK9HDuK713cq
!curl -o Teste4.png https://drive.google.com/uc?export=1ZzcgxptDJ95hr8urx32rFYufphDgetNo
!curl -o Teste5.png https://drive.google.com/uc?export=1yI6bkep_txNnXuXBBzGHX-YzkZfHhqp-
!curl -o Teste6.png https://drive.google.com/uc?export=1DocxezhSIab4sWlnavNlzfD03ixgytJn



  Using cached google_generativeai-0.6.0-py3-none-any.whl (158 kB)
  Using cached google_ai_generativelanguage-0.6.4-py3-none-any.whl (679 kB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.4.0
    Uninstalling google-ai-generativelanguage-0.4.0:
      Successfully uninstalled google-ai-generativelanguage-0.4.0
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.3.0
    Uninstalling google-generativeai-0.3.0:
      Successfully uninstalled google-generativeai-0.3.0


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
  Using cached reportlab-4.2.0-py3-none-any.whl (1.9 MB)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1685    0  1685    0     0   4170      0 --:--:-- --:--:-- --:--:--  4181
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1685    0  1685    0     0   7216      0 --:--:-- --:--:-- --:--:--  7200
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1685    0  1685    0     0   1645      0 --:--:--  0:00:01 --:--:--  1647
  % T

In [ ]:
import google.generativeai as genai
from google.colab import files
from IPython.display import display
from IPython.display import Markdown
import PIL.Image
import pathlib
import os
from tqdm import tqdm
from reportlab.pdfgen import canvas
import re

global files_gemini
global answer
from google.colab import userdata

# !curl -o questao_listaUNb.png https://drive.google.com/uc?export=view&id=15DZT3vHPOOKs06cyPUWjATMnq9aljf8u
# !curl -o questão3.png https://drive.google.com/uc?export=view&id=13Bt0G4MNHn_yFiDAMLRn2MpAUFxqGwmx


# ------------ MODELO------------------
GOOGLE_AI_STUDIO = userdata.get('google_IA_Studio')
genai.configure(api_key=GOOGLE_AI_STUDIO)

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

# Create the model
# See https://ai.google.dev/api/python/google/generativeai/GenerativeModel
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)

# TODO Make these files available on the local file system
# You may need to update the file paths
files_gemini = [
  upload_to_gemini("questão3.png", mime_type="image/png"),
  upload_to_gemini("questao_listaUNb.png", mime_type="image/png"),
]

len_files_gemini = len(files_gemini)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        files_gemini[0],
        "Escreva a forma que podemos ler oralmente as equações das questões. Não mostre visualmente as equações e leia por extenso todos os operadores, não basta escrever. ",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Questão 1 - Para cada função f de x, determine a derivada f linha de x no ponto x zero indicado.\na) f de x igual a x ao quadrado\nb) f de x igual a 2x mais 3\nc) f de x igual a menos 3x\nd) f de x igual a x ao quadrado menos 3x\ne) f de x igual a x ao quadrado menos 4\nf) f de x igual a 1 dividido por x\ng) f de x igual a 1 dividido por x ao quadrado\nh) f de x igual a x ao quadrado menos 3x mais 4\n\nQuestão 2 - Dada a função f de x igual a x se x for menor ou igual a 1 e 2 se x for maior que 1. Mostre que não existe f linha de 1.\n\nQuestão 3 - Considere a função f de x igual a módulo de 2x. Mostre que não existe f linha de 0.\n\nQuestão 4 - Obtenha a derivada de cada função a seguir.\na) f de x igual a 10\nb) f de x igual a x elevado a 5\nc) f de x igual a 10 elevado a 5\nd) f de x igual a 1 dividido por x ao quadrado\ne) f de x igual a x ao quadrado mais x ao cubo\nf) f de x igual a 10x ao cubo mais 5x ao quadrado\ng) f de x igual a 2x mais 1\nh) f de x igual a 3x ao quadrado menos 6x mais 10\ni) f de x igual a 5u ao cubo menos 2u ao quadrado mais 6u mais 7\nj) f de x igual a 3 ln de x mais 5\nk) f de x igual a 10 ln de x menos 3x mais 6\nl) f de x igual a 5 sen de x mais 2 cos de x menos 4\nm) f de x igual a x vezes sen de x\nn) f de x igual a x ao quadrado vezes ln de x\no) f de x igual a 2x ao quadrado menos 3x mais 5 vezes 2x menos 1\np) f de x igual a sen de x dividido por x ao quadrado\nq) f de x igual a tg de x vezes sen de x dividido por cos de x\nr) f de x igual a x menos 1 dividido por x ao quadrado menos 2\ns) f de x igual a 2 dividido por x ao cubo mais 5 dividido por x ao quadrado\nt) f de x igual a x ao cubo mais 1 dividido por x ao cubo mais 1\nu) f de x igual a x elevado a 1 terço mais x elevado a 1 quarto\nv) f de x igual a raiz quadrada de 3x mais raiz quadrada de 5x mais 10\nw) f de x igual a raiz quadrada de x vezes sen de x\nx) f de x igual a ln de x dividido por raiz quadrada de x",
      ],
    },
    {
      "role": "user",
      "parts": [
        "leia por extenso todos os operadores, não basta escrever. ",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Questão 1 - Para cada função f de x, determine a derivada de f de x em relação a x no ponto x zero indicado.\na) f de x igual a x elevado ao quadrado.\nb) f de x igual a 2 vezes x mais 3.\nc) f de x igual a menos 3 vezes x.\nd) f de x igual a x elevado ao quadrado menos 3 vezes x.\ne) f de x igual a x elevado ao quadrado menos 4.\nf) f de x igual a 1 dividido por x.\ng) f de x igual a 1 dividido por x elevado ao quadrado.\nh) f de x igual a x elevado ao quadrado menos 3 vezes x mais 4.\n\nQuestão 2 - Dada a função f de x igual a x se x for menor ou igual a 1 e 2 se x for maior que 1. Mostre que não existe a derivada de f de x em relação a x para x igual a 1.\n\nQuestão 3 - Considere a função f de x igual ao módulo de 2 vezes x. Mostre que não existe a derivada de f de x em relação a x para x igual a 0.\n\nQuestão 4 - Obtenha a derivada de cada função a seguir.\na) f de x igual a 10.\nb) f de x igual a x elevado à quinta potência.\nc) f de x igual a 10 elevado à quinta potência.\nd) f de x igual a 1 dividido por x elevado ao quadrado.\ne) f de x igual a x elevado ao quadrado mais x elevado ao cubo.\nf) f de x igual a 10 vezes x elevado ao cubo mais 5 vezes x elevado ao quadrado.\ng) f de x igual a 2 vezes x mais 1.\nh) f de x igual a 3 vezes x elevado ao quadrado menos 6 vezes x mais 10.\ni) f de x igual a 5 vezes u elevado ao cubo menos 2 vezes u elevado ao quadrado mais 6 vezes u mais 7.\nj) f de x igual a 3 vezes o logaritmo natural de x mais 5.\nk) f de x igual a 10 vezes o logaritmo natural de x menos 3 vezes x mais 6.\nl) f de x igual a 5 vezes o seno de x mais 2 vezes o cosseno de x menos 4.\nm) f de x igual a x vezes o seno de x.\nn) f de x igual a x elevado ao quadrado vezes o logaritmo natural de x.\no) f de x igual a 2 vezes x elevado ao quadrado menos 3 vezes x mais 5 vezes 2 vezes x menos 1.\np) f de x igual ao seno de x dividido por x elevado ao quadrado.\nq) f de x igual à tangente de x vezes o seno de x dividido pelo cosseno de x.\nr) f de x igual a x menos 1 dividido por x elevado ao quadrado menos 2.\ns) f de x igual a 2 dividido por x elevado ao cubo mais 5 dividido por x elevado ao quadrado.\nt) f de x igual a x elevado ao cubo mais 1 dividido por x elevado ao cubo mais 1.\nu) f de x igual a x elevado a 1 terço mais x elevado a 1 quarto.\nv) f de x igual à raiz quadrada de 3 vezes x mais a raiz quadrada de 5 vezes x mais 10.\nw) f de x igual à raiz quadrada de x vezes o seno de x.\nx) f de x igual ao logaritmo natural de x dividido pela raiz quadrada de x. \n",
      ],
    },
    {
      "role": "user",
      "parts": [
        files_gemini[1],
        "faça o mesmo para essa outra questão",
      ],
    },
    {
      "role": "model",
      "parts": [
        "1) Supondo que y é igual a y de u e u é igual a u de x, use a regra da cadeia para calcular a derivada de y em relação a x nos itens abaixo.\na) y igual a u elevado à quarta potência mais 1; u igual a 3 vezes x ao quadrado menos 2 vezes x.\nb) y igual à raiz quadrada de u; u igual a 1 dividido por (x menos 1).\nc) y igual a u ao quadrado mais 2 vezes u menos 3; u igual à raiz quadrada de x.\nd) y igual a u ao cubo mais u; u igual a 1 dividido pela raiz quadrada de x.\ne) y igual ao cosseno de u; u igual a x mais x ao quadrado.\nf) y igual ao seno de u; u igual à raiz quadrada de x. \n",
      ],
    },
  ]
)



# -------------------FIM DO MODELO------------------------------


# -------------------RECEBE ARQUIVO------------------------------
#faz upload dos arquivos
uploaded = files.upload()
uploaded = list(uploaded.keys())
nome_do_arquivo = uploaded[0]

#renomeia o pdf enviado para poder trabalhar com ele
old_name = '/content/'+nome_do_arquivo
new_name = r"/content/pdfAntigo.pdf"
os.rename(old_name, new_name)

#cria a pasta output que vai conter as imagens das páginas do pdf
!mkdir output
!pdftoppm '/content/pdfAntigo.pdf' -f {1} -l {-1} output/images -jpeg

#acha o nome das imagens contidas na pasta output
def find_str_img_names():
  paths = !ls output
  paths = list(paths)
  nomes = []
  for i in paths:
    for j in i.split():
      nomes.append(j)
  return sorted(nomes)

#acha os paths das imagens em output
def find_uploaded_img_paths():
  nomes = list(find_str_img_names())
  paths = []
  for i in nomes:
    paths.append("/content/output/"+i)
  return paths

#manda as imagens pro IAstudio e coloca na lista files_gemini
def sendo_to_IA(paths):
  for i in paths:
    files_gemini.append(upload_to_gemini(i, mime_type="image/png"))

#com as imagens enviadas, utiliza-se da pergunta (variável "prompt_used") para perguntar a leitura ao chat
def ask_page_by_page(prompt_used):
  pages_answered = []
  for i in range(len_files_gemini):
    files_gemini.pop(0)

  for i in range(len(files_gemini)):
    response = model.generate_content([prompt_used,files_gemini[i]])
    response.resolve()
    pages_answered.append(response.text)
  return pages_answered

#define a pergunta usada no chat
prompt_used= "Escreva a forma que podemos ler oralmente as equações de toda a página dessa imagem específica para pessoas cegas. Separe em tópicos. Não mostre visualmente as equações e leia por extenso todos os operadores, não basta escrever."
# prompt_used = "Você será um assistente de leitura para pessoas com deficiência visual, transcrevendo exercícios de matemática de imagens para texto. Sua tarefa é converter os enunciados e as equações das imagens para texto, garantindo clareza e precisão. As equações devem ser transcritas da mesma forma que seriam lidas em voz alta. Siga as diretrizes a seguir:\n\n##Enunciados\n-Transcreva tudo que for texto no exercício exatamente como ele aparece.\n-Se houver alguma formula matemática no meio enunciado transcreva assim como uma equação no mesmo ponto que estava no texto \n-Não deixe de pôr o indicador de tópico dos enunciados nem o indicador dos subtópicos do exercício.\n-Transcreva os tópicos na ordem que eles aparecem na lista, mas mantenha os subtópicos de um exercício transcritos em ordem alfabética ou numérica.\n\n###Exemplo da estrutura da transcrição:\n-Exercício da imagem:\n1. *texto do enunciado\na)equação*          b)*equação* \nc)*equação*          d)*equação* \ne)*equação\n2. **texto do enunciado\n\n-Transcrição:\n1. **texto do enunciado\na)Transcrição da equação\nb)Transcrição da equação\nc)Transcrição da equação\nd)Transcrição da equação\ne)Transcrição da equação\n\n2. **texto do enunciado*\n\n\n##Equações\n-Transcreva as equações de forma clara e sem ambiguidade.\n-Poderá ser utilizado (),{} e [] para evitar ambiguidades nas transcrições, seguindo sempre essa ordem de mais interno para mis externo.\n- Use a notação matemática de forma descritiva.\n-Use como base para o entendimento visual da equação a linguagem LaTex, mas sua transcrição deve sempre ser para texto cursivo, “faladamente”.\n\n###Exemplo da transcrição de equações: será utilizado equações em LaTex para representar o que será uma equação na imagem\nEquação: \\( e^{x+2} * y \\)\nTranscrição: Exponencial de( x mais 2) vezes y.\n\nEquação: \\( e^{x}+2 * y \\)\nTranscrição: Exponencial de( x )mais 2 y.\n\nEquação: \\( f(x) = \\frac{x^4}{x^2 + x^3} \\)\nTranscrição: f de x é igual a x a quarta dividido por (x ao quadrado mais x ao cubo).\n\nEquação: \\(\\( f(x) = \\frac{x^4}{x^2}+ x^3  \\)\nTranscrição: f de x é igual a x a quarta dividido por (x ao quadrado) mais x ao cubo."
#manda pra IA, com auxilio das funções acima
sendo_to_IA(find_uploaded_img_paths())

#define a variável de páginas em forma oral
pages_answered = []
pages_answered = ask_page_by_page(prompt_used)
print(pages_answered)

#---------------------------- cria novo pdf -------------------

#adapta novo título para "nome_antigo_adaptado.pdf"
def adapt_new_title(old_title):
  if ".pdf" in old_title:
    old_title = old_title.replace(".pdf","")
  return old_title

str_new_file_name = adapt_new_title(uploaded[0])+"_adaptado.pdf"

new_file_name = canvas.Canvas(str_new_file_name)

#onde o código pega o primeiro elemento da lista answer, a qual é criadaa partir das respostas da IA
#Este primeiro elemento é a resposta do chat para a primeira página do pdf e é referenciada na hora de criar um novo pdf

characters = 92 #Número de caracteres em uma linha
def split_lines(content_in_the_page):
    lines = content_in_the_page.split('\n')
    return lines

def split_words(content, characters):
    words =content.split(' ')
    Line1= ''
    Line2= ''
    Ver = False
    for word in words:
        if Ver == False:
            if len(Line1) + len(word) +1 <= characters:
                Line1 = Line1 + word + ' '
            else:
                Ver = True
                Line2 = Line2 + word + ' '
        else:
            Line2 = Line2 + word + ' '
    return Line1, Line2
def verify_lines(lines:list, characters:int) ->list:
    new_lines = []
    for line in lines:
        while len(line) > characters:
            Linha1, Linha2 =split_words(line, characters)
            new_lines.append(Linha1)
            line = Linha2
        new_lines.append(line)
    return new_lines
#cria um novo pdf
def create_new_PDF(new_file_name, lines):
    Counter = 0
    for j in range(len(lines)):
      j2 = j -51*Counter
      if j2 == 51:
        Counter = Counter + 1
        j2 = j2 -51
        new_file_name.showPage()
        new_file_name.drawString(1, 800, '.')
      line = 800-15*j2
      new_file_name.drawString(20,line, lines[j] )


lines = []

for i in range(len(pages_answered)):
  separated_lines = split_lines(pages_answered[i])
  for j in separated_lines:
    if "#" in j :
      j = j.replace("#","")
    if "*" in j:
      j = j.replace("*","")
    lines.append(j)

new_lines = verify_lines(lines, characters)
new_lines = [s for s in new_lines if s != '']
new_lines = [s for s in new_lines if s != ' ']

create_new_PDF(new_file_name, new_lines)
new_file_name.save()


path = '/content/'+str_new_file_name
files.download(path)





Uploaded file 'questão3.png' as: https://generativelanguage.googleapis.com/v1beta/files/c43evtvb48pd
Uploaded file 'questao_listaUNb.png' as: https://generativelanguage.googleapis.com/v1beta/files/r7qvgmbtub2z


KeyboardInterrupt: 

In [ ]:
for i in pages_answered:
  print(i)

## MAT-111 - Cálculo Diferencial e Integral I
Bacharelado em Matemática - 2010

## 1ª Lista de exercícios
### 1. Limite de funções
1. Calcule os seguintes limites, caso existam:
    1. Limite quando x tende a menos 3 de 2x ao cubo mais 9x ao quadrado mais 12x mais 4, sobre x ao cubo menos 2x ao quadrado mais 4x mais 8
    2. Limite quando x tende a menos 3 de raiz quadrada de x ao quadrado mais 16 menos 5, sobre x ao quadrado mais 3x
    3. Limite quando x tende a menos 2 de raiz quadrada de x ao quadrado mais 12 menos 4, sobre 2 menos raiz cúbica de x ao cubo menos 4
    4. Limite quando x tende a menos 1 de raiz quadrada de 2x menos 1, sobre raiz quadrada de x menos 1
    5. Limite quando x tende a 0 de raiz quadrada de x ao quarto mais 1, sobre x ao quarto mais 1
    6. Limite quando x tende a 1 de raiz quadrada de x menos 1, sobre x menos 1
    7. Limite quando x tende a 1 de raiz quadrada de x ao quadrado menos 1, mais raiz quadrada de x menos 1, sobre raiz quadrada de x menos 1
 

In [ ]:

# !curl -o questão3.png https://drive.google.com/uc?export=view&id=13Bt0G4MNHn_yFiDAMLRn2MpAUFxqGwmx
# !curl -o Teste1.png https://drive.google.com/uc?export=18GMKkBPAx5zqpC_LtsJ-mtX0K3Hd6Pfk

GOOGLE_AI_STUDIO = userdata.get('google_IA_Studio')
genai.configure(api_key=GOOGLE_AI_STUDIO)

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

# Create the model
# See https://ai.google.dev/api/python/google/generativeai/GenerativeModel
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)

# TODO Make these files_gemini available on the local file system
# You may need to update the file paths
files_gemini = [
  upload_to_gemini("Teste1.png", mime_type="image/png"),
  upload_to_gemini("Teste2.png", mime_type="image/png"),
]

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        files_gemini[0],
        "Leia por extenso a questão e  todas as equações matemáticas presentes na imagem seguindo esse passos: Diga cada símbolo explicitamente, descreva todas as operações que aparecem na equação e indique claramente onde começam e terminam os parênteses\n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "A imagem apresenta uma questão de limite de funções com seis equações matemáticas. \n\n**Questão:**  Calcule os seguintes limites, caso existam:\n\n**Equação 1:** limite, quando x tende a menos dois, da fração, cujo numerador é dois x ao cubo mais nove x ao quadrado mais doze x mais quatro, e o denominador é menos x ao cubo menos dois x ao quadrado mais quatro x mais oito.\n\n**Equação 2:** limite, quando x tende a menos três, da fração, cujo numerador é raiz quadrada de x ao quadrado mais dezesseis menos cinco, e o denominador é x ao quadrado mais três x.\n\n**Equação 3:** limite, quando x tende a dois, da fração, cujo numerador é raiz quadrada de x ao quadrado mais doze menos quatro, e o denominador é dois menos raiz cúbica de x ao cubo menos quatro.\n\n**Equação 4:** limite, quando x tende a um meio, da fração, cujo numerador é raiz quadrada de dois x menos um, e o denominador é raiz quadrada de dois x menos um.\n\n**Equação 5:** limite, quando x tende a zero, da fração, cujo numerador é raiz quinta de x à quarta mais um menos um, e o denominador é x à quarta.\n\n**Equação 6:** limite, quando x tende a um, da fração, cujo numerador é x menos um, e o denominador é raiz cúbica de x menos um. \n",
      ],
    },
    {
      "role": "user",
      "parts": [
        files_gemini[1],
        "Leia por extenso a questão e todas as equações matemáticas presentes na imagem seguindo esse passos: Diga cada símbolo explicitamente, descreva todas as operações que aparecem na equação e indique claramente onde começam e terminam os parênteses",
      ],
    },
    {
      "role": "model",
      "parts": [
        "A imagem apresenta duas questões matemáticas. \n\n**Questão 1:** Usando a definição formal, encontre as derivadas das funções a seguir.\n\n**Equação 1.a:** y igual a menos dois x mais sete.\n\n**Equação 1.b:** x ao quadrado menos cinco x mais sete.\n\n**Equação 1.c:** y igual a dois sobre x. \n\n**Equação 1.d:** raiz quadrada de três x mais um.\n\n**Equação 1.e:** um sobre raiz quadrada de x mais um.\n\n\n**Questão 2:** (Flemming e Gonçalves, pág. 127) Dadas as funções f de x igual a cinco menos dois x e g de x igual a três x ao quadrado menos um, determinar:\n\n**Equação 2.a:** f linha de um mais g linha de um.\n\n**Equação 2.b:** duas vezes f linha de zero menos g linha de menos dois.\n\n**Equação 2.c:** f de dois menos f linha de menos dois.\n\n**Equação 2.d:** abre colchetes, g linha de zero, fecha colchetes, ao quadrado mais um meio de g linha de zero mais g de zero.\n\n**Equação 2.e:** f de cinco sobre dois, menos, abre parênteses, f linha de cinco sobre dois sobre g linha de cinco sobre dois, fecha parênteses.\n\n\n**Resposta:**\n\n**Resposta 2.a:** quatro.\n\n**Resposta 2.b:** oito.\n\n**Resposta 2.c:** três.\n\n**Resposta 2.d:** menos um.\n\n**Resposta 2.e:** dois sobre quinze. \n",
      ],
    }
  ]
)

response = model.generate_content(["Escreva a forma que podemos ler oralmente as equações da última imagem enviada. Não mostre visualmente as equações e leia por extenso todos os operadores, não basta escrever.",files_gemini[0]])
response.resolve()
print(response.text)

Uploaded file 'Teste1.png' as: https://generativelanguage.googleapis.com/v1beta/files/zhbism9pl6re
Uploaded file 'Teste2.png' as: https://generativelanguage.googleapis.com/v1beta/files/vh91j0mbihkw


ERROR:tornado.access:500 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 800.45ms


InternalServerError: 500 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting

SyntaxError: unterminated string literal (detected at line 1) (<ipython-input-95-711718c2656c>, line 1)